In [1]:
import pandas as pd

In [2]:
data_session_1=pd.read_json('assets/prodigy_sessions/el_session_1.jsonl',lines=True)
data_session_2=pd.read_json('assets/prodigy_sessions/el_session_2.jsonl',lines=True)

In [3]:
data=data_session_1.append(data_session_2).reset_index()

In [4]:
data[data.drop(['flagged','url','gu_url'],1).isna().any(1)]

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,index,text,_input_hash,_task_hash,gu_url,spans,meta,_is_binary,options,config,_view_id,accept,answer,_timestamp,_annotator_id,_session_id,flagged,url


In [5]:
data['_input_hash'].value_counts()

 1181780303    17
-1155515297    13
-122158038     11
-801910587     11
-1692783485    11
               ..
 811129546      1
-383857937      1
 925792991      1
 1934483787     1
-1462095615     1
Name: _input_hash, Length: 1252, dtype: int64

In [6]:
data.shape

(2906, 18)

In [8]:
data['text'].nunique()

1258

In [10]:
def find_no_candidates(row, no_candidate_sub_str=['NEL','NER']):
    return any(sub_str in ' '.join(row['accept']) for sub_str in no_candidate_sub_str)

In [11]:
data=data[~data.apply(find_no_candidates,1)]

In [ ]:
data['accept_len']=data['accept'].apply(lambda x: len(''.join(x))).sort_values()

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data.shape

(1649, 19)

In [ ]:
# Number of empty accepts
data[data['accept_len']==0].shape

(3, 19)

In [ ]:
data=data[data['accept_len']!=0]

In [ ]:
data.shape

(1646, 19)

In [17]:
def find_best_candidate(row,best_candidate=0):
    best_candidate_d={}
    for option_id in row['accept']:
        for candidate in list(filter(lambda x: option_id == x['id'], row['options'])):
            candidate_len=len(candidate['html'].split('a>:')[1])
            best_candidate_d[option_id]=candidate_len
    return [max(best_candidate_d, key=best_candidate_d.get)][0]

In [18]:
data['accept']=data.apply(find_best_candidate,1)

In [19]:
data['mentions']=data['spans'].apply(lambda x: x[0]['text'])

In [26]:
data['mentions'].value_counts()

Donald Trump         70
Boris Johnson        48
Corbyn               42
Theresa May          38
Trump                38
                     ..
Alfred Hitchcock      1
Charlie Chaplin       1
Brigitte Bardot       1
Hank Meijer           1
James Brokenshire     1
Name: mentions, Length: 888, dtype: int64

In [20]:
pd.DataFrame(data['mentions'].value_counts()).to_csv('annotations_2022_11_08.csv')